In [ ]:
import requests
import time
from notebookutils import mssparkutils
from datetime import datetime

# Set the workspace id, report id and export format
workspace_id = "912e6fbb-75f5-40b4-ac17-7ea2a116e9b2"
report_id = "8832f89e-9610-4e10-a047-4ffbda6018ad"
export_format="PDF"

# Different than displayname, either use the GetPages api or copy from the url when the desired page is selected
page_name="ReportSection"

# Set additional export configuration
export_config = {
    "format": export_format,
    "powerBIReportConfiguration" :{
        "pages": [
            {"pageName": page_name}
        ],
        "reportLevelFilters": [
            {"filter":"Department/VP eq 'Carl Morris'"}
        ]
    },
}

# Set the header with the authentication token
auth_token = mssparkutils.credentials.getToken('pbi')
headers = {
    "Authorization": f"Bearer {auth_token}",
    "Content-Type": "application/json"
}


In [ ]:
# Schedule the export job in Power BI
export_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/reports/{report_id}/ExportTo"
response = requests.post(
    url= export_url,
    headers=headers,
    json=export_config
)

# Check the response
if response.status_code == 202:
    print("Export request submitted successfully.")
    print (response.content)
else:
    print("Export request failed with status code:", response.status_code)
    print("Error message:", response.text)

In [ ]:
# Check the export status periodically until completed
export_id =  response.json()["id"]
check_export_status_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/reports/{report_id}/exports/{export_id}"
while True:
    response = requests.get(
        url=check_export_status_url,
        headers=headers
    )
    print(response.content)
    export_status = response.json()
    if export_status["status"] == "Succeeded":
        print("Export completed successfully.")
        break
    elif export_status["status"] in ["Failed", "Cancelled"]:
        print("Export failed or was cancelled.")
        break

    print("Export status:", export_status["status"])
    time.sleep(5)  # Wait for 5 seconds before checking again

In [ ]:
# get the content and save it to onelake
if export_status["status"] == "Succeeded":
    content_url=export_status["resourceLocation"]
    report_name=export_status["reportName"]
    report_extension=export_status["resourceFileExtension"]
    response = requests.get(
            url=content_url,
            headers=headers
        )
    # create directory
    directory = "Files/ExportedFiles"
    mssparkutils.fs.mkdirs(directory) 

    # build the path to store the file
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M")
    filename = f"{formatted_datetime}_{report_name}{report_extension}"
    full_file_path = mssparkutils.fs.getMountPath(f"/default/{directory}/") + filename

    #save file
    with open(full_file_path, "wb") as file:
        file.write(response.content)